# Import Library

In [ ]:
import pandas as pd
import json
import os
import torch
import re
import ast

from scipy.spatial.distance import cosine
from torch import nn
from openai import OpenAI
from matplotlib import pyplot as plt
from transformers import GPT2Tokenizer
from sentence_transformers import SentenceTransformer
from sklearn.model_selection import train_test_split


# Persiapan Dataset

## Load Data

In [ ]:
# Load Data
data = pd.read_csv('putusan_data.csv')

In [ ]:
# Menampilkan Preview Data
data.head()

,No Putusan,Lembaga Peradilan,Barang Bukti,Amar Putusan
0,640/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah bekas bungkus rokok Luxio berisi...,M E N G A D I L I :; Menyatakan Terdakwa RAKA ...
1,641/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket tembakau Gorila yang dibungkus ...,M E N G A D I L I; Menyatakan Terdakwa I. AHMA...
2,545/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas pinggang warna hitam;; 1 (sa...,MENGADILI:; Menyatakan terdakwa AKIN SUNARYO B...
3,586/Pid.Sus/2024/PN Smn,PN SLEMAN,Bekas bungkus rokok merk Camel yang di dalamny...,MENGADILI:; Menyatakan Terdakwa Septian Eko Ca...
4,446/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas ransel warna hitam bertulisk...,M E N G A D I L I; Menyatakan Terdakwa I. Cost...


## Preprocessing

In [ ]:
# Merubah nama dataset hingga tidak ada spasi
data.rename(columns={"No Putusan": "no_putusan", "Lembaga Peradilan": "lembaga_peradilan", "Barang Bukti": "barang_bukti","Amar Putusan": "amar_putusan"}, inplace=True)

In [ ]:
# Melakukan Pre-processing data teks
def clean_column(data, column_name):

    data[column_name] = data[column_name].astype(str)  # Memastikan Kolom bertipe string
    data[column_name] = (
        data[column_name]
        .str.replace('\n', ' ', regex=True)  # Menghilangkan baris baru
        .str.replace('\r', '', regex=True)  # Remove carriage returns
        .str.replace('\s+', ' ', regex=True)  # Menghilangkan spasi berulang
        .str.strip()  # Menghapus spasi di depan dan belakang kalimat
    )
    return data

In [ ]:
data = clean_column(data, 'barang_bukti')
data = clean_column(data, 'amar_putusan')

In [ ]:
# Inisialisasi tokenaizer gpt2
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")

tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

In [ ]:
# Melakukan perhitungan tokenizer
def count_tokens(text: str) -> int:
    return len(tokenizer.encode(text))

In [ ]:
# Memastikan kolom amar_putusan tersedia
if 'amar_putusan' in data.columns:
    # Mengisi null value jika ada
    data['amar_putusan'] = data['amar_putusan'].fillna('').astype(str)

    # Membuat kolom baru berisi token untuk amar_putusan
    data['tokens'] = data['amar_putusan'].apply(count_tokens)
else:
    print("Error: Column 'amar_putusan' not found in DataFrame") # jika amar_putusan tidak ada dalam dataframe

Token indices sequence length is longer than the specified maximum sequence length for this model (1188 > 1024). Running this sequence through the model will result in indexing errors


In [ ]:
# Menambahkan konteks
data['Context'] = "Berdasarkan barang bukti berikut: " + data.barang_bukti + " Diputuskan oleh hakim: " + data.amar_putusan

In [ ]:
# Menggunakan hanya 50 data dalam dataset
data = data.head(50)

## Hasil Persiapan Data

In [ ]:
# Hasil data setelah diproses
data

,no_putusan,lembaga_peradilan,barang_bukti,amar_putusan,tokens,Context
0,640/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah bekas bungkus rokok Luxio berisi...,M E N G A D I L I :; Menyatakan Terdakwa RAKA ...,376,Berdasarkan barang bukti berikut: 1 (satu) bua...
1,641/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket tembakau Gorila yang dibungkus ...,M E N G A D I L I; Menyatakan Terdakwa I. AHMA...,1188,Berdasarkan barang bukti berikut: 1 (satu) pak...
2,545/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas pinggang warna hitam;; 1 (sa...,MENGADILI:; Menyatakan terdakwa AKIN SUNARYO B...,465,Berdasarkan barang bukti berikut: 1 (satu) bua...
3,586/Pid.Sus/2024/PN Smn,PN SLEMAN,Bekas bungkus rokok merk Camel yang di dalamny...,MENGADILI:; Menyatakan Terdakwa Septian Eko Ca...,699,Berdasarkan barang bukti berikut: Bekas bungku...
4,446/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas ransel warna hitam bertulisk...,M E N G A D I L I; Menyatakan Terdakwa I. Cost...,2388,Berdasarkan barang bukti berikut: 1 (satu) bua...
5,553/Pid.Sus/2024/PN Smn,PN SLEMAN,: - 1 (satu) buah tempat kaca mata berwarna hi...,M E N G A D I L I; 1. Menyatakan Terdakwa REDI...,917,Berdasarkan barang bukti berikut: : - 1 (satu)...
6,552/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) Plastik paket sabu yang diselipkan di...,M E N G A D I L I; 1. Menyatakan Terdakwa MUHL...,1245,Berdasarkan barang bukti berikut: 1 (satu) Pla...
7,551/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) plastic klip berisi tembakau gorilla ...,M E N G A D I L I; Menyatakan Terdakwa Guntur ...,539,Berdasarkan barang bukti berikut: 1 (satu) pla...
8,550/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) bungkus rokok CAMEL yang berisi 1 (sa...,M E N G A D I L I; ; Menyatakan Terdakwa ROCNA...,616,Berdasarkan barang bukti berikut: 1 (satu) bun...
9,548/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket Shabu ukuran kecil dibungkus de...,M E N G A D I L I :; ; Menyatakan TerdakwaDWI ...,407,Berdasarkan barang bukti berikut: 1 (satu) pak...


In [ ]:
# Menyimpan data yang telah diproses
data.to_csv('processed_data.csv', index=False)

# Merancang Model QA

## Inisialisasi Open AI

In [ ]:
# Inisialisasi API Key (ganti dengan API key yang dimiliki)
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
client = OpenAI()

In [ ]:
# Penggunaan fitur chat.completions

completion = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {
            "role": "user",
            "content": "buatkan pertanyaan tentang pidana kasus narkotika"
        }
    ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content='Tentu, berikut adalah beberapa pertanyaan yang dapat diajukan terkait dengan kasus pidana narkotika:\n\n1. Apa saja jenis-jenis narkotika yang diatur dalam undang-undang pidana di Indonesia?\n2. Bagaimana proses hukum yang dilakukan oleh aparat penegak hukum ketika menemukan kasus penyalahgunaan narkotika?\n3. Apa perbedaan antara pengguna, pengedar, dan bandar narkotika dalam konteks hukum pidana?\n4. Apa saja sanksi yang dapat dijatuhkan kepada pelanggar undang-undang narkotika sesuai dengan Pasal 111 hingga Pasal 132 Undang-Undang No. 35 Tahun 2009?\n5. Bagaimana bukti-bukti yang diperlukan dalam kasus pidana narkotika untuk membuktikan bahwa seseorang adalah pengedar atau pengguna?\n6. Apa peran rehabilitasi dalam penanganan kasus narkotika, dan bagaimana prosesnya diatur dalam hukum?\n7. Apakah ada perbedaan dalam penanganan kasus narkotika untuk anak di bawah umur dibandingkan dengan pelaku dewasa?\n8. Bagaimana dampak sosial dan ekonomi dari kasus 

## Implementasi Open AI kasus

In [ ]:
df = pd.read_csv('processed_data.csv')

In [ ]:
df.head()

,no_putusan,lembaga_peradilan,barang_bukti,amar_putusan,tokens,Context
0,640/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah bekas bungkus rokok Luxio berisi...,M E N G A D I L I :; Menyatakan Terdakwa RAKA ...,376,Berdasarkan barang bukti berikut: 1 (satu) bua...
1,641/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket tembakau Gorila yang dibungkus ...,M E N G A D I L I; Menyatakan Terdakwa I. AHMA...,1188,Berdasarkan barang bukti berikut: 1 (satu) pak...
2,545/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas pinggang warna hitam;; 1 (sa...,MENGADILI:; Menyatakan terdakwa AKIN SUNARYO B...,465,Berdasarkan barang bukti berikut: 1 (satu) bua...
3,586/Pid.Sus/2024/PN Smn,PN SLEMAN,Bekas bungkus rokok merk Camel yang di dalamny...,MENGADILI:; Menyatakan Terdakwa Septian Eko Ca...,699,Berdasarkan barang bukti berikut: Bekas bungku...
4,446/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas ransel warna hitam bertulisk...,M E N G A D I L I; Menyatakan Terdakwa I. Cost...,2388,Berdasarkan barang bukti berikut: 1 (satu) bua...


### Generate pertanyaan

In [ ]:
def get_questions(context):
    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "Kamu adalah asisten saya untuk membuat pertanyaan dalam konteks hukum pidana narkotika."},
                {
                    "role": "user",
                    "content": f"Tuliskan 3 (tiga) pertanyaan dalam bahasa indonesia berdasakan teks dibawah ini:\n\nText: {context}\n\nQuestions:\n"}
            ],
            model="gpt-4o-mini",
            temperature=1,
            logprobs=True,
            top_logprobs=10,
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")

df['questions']= df.Context.apply(get_questions)
df.head()
print(df[['questions']].values[0][0])

1. Apa barang bukti yang ditemukan dalam kasus Terdakwa RAKA ARDI SULISTYO dan berapa berat total narkotika yang terlibat?
   
2. Apa keputusan hakim mengenai hukuman yang dijatuhkan kepada Terdakwa dan apakah masa penahanannya dikurangkan dari pidana yang dijatuhkan?

3. Apa saja barang bukti yang dirampas dan dimusnahkan berdasarkan putusan hakim dalam kasus ini?


In [ ]:
df

,no_putusan,lembaga_peradilan,barang_bukti,amar_putusan,tokens,Context,questions
0,640/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah bekas bungkus rokok Luxio berisi...,M E N G A D I L I :; Menyatakan Terdakwa RAKA ...,376,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa barang bukti yang ditemukan dalam kasus...
1,641/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket tembakau Gorila yang dibungkus ...,M E N G A D I L I; Menyatakan Terdakwa I. AHMA...,1188,Berdasarkan barang bukti berikut: 1 (satu) pak...,1. Apa dasar hukum yang menjadi acuan dalam me...
2,545/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas pinggang warna hitam;; 1 (sa...,MENGADILI:; Menyatakan terdakwa AKIN SUNARYO B...,465,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa dasar keputusan hakim dalam menyatakan ...
3,586/Pid.Sus/2024/PN Smn,PN SLEMAN,Bekas bungkus rokok merk Camel yang di dalamny...,MENGADILI:; Menyatakan Terdakwa Septian Eko Ca...,699,Berdasarkan barang bukti berikut: Bekas bungku...,1. Apa jenis narkotika yang ditemukan dalam ba...
4,446/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas ransel warna hitam bertulisk...,M E N G A D I L I; Menyatakan Terdakwa I. Cost...,2388,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa alasan hakim memutuskan untuk menjatuhk...
5,553/Pid.Sus/2024/PN Smn,PN SLEMAN,: - 1 (satu) buah tempat kaca mata berwarna hi...,M E N G A D I L I; 1. Menyatakan Terdakwa REDI...,917,Berdasarkan barang bukti berikut: : - 1 (satu)...,1. Apa saja jenis barang bukti yang ditemukan ...
6,552/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) Plastik paket sabu yang diselipkan di...,M E N G A D I L I; 1. Menyatakan Terdakwa MUHL...,1245,Berdasarkan barang bukti berikut: 1 (satu) Pla...,1. Apa dasar hukum yang digunakan oleh hakim d...
7,551/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) plastic klip berisi tembakau gorilla ...,M E N G A D I L I; Menyatakan Terdakwa Guntur ...,539,Berdasarkan barang bukti berikut: 1 (satu) pla...,1. Apa saja barang bukti yang ditemukan dalam ...
8,550/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) bungkus rokok CAMEL yang berisi 1 (sa...,M E N G A D I L I; ; Menyatakan Terdakwa ROCNA...,616,Berdasarkan barang bukti berikut: 1 (satu) bun...,1. Apa saja barang bukti yang ditemukan dalam ...
9,548/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket Shabu ukuran kecil dibungkus de...,M E N G A D I L I :; ; Menyatakan TerdakwaDWI ...,407,Berdasarkan barang bukti berikut: 1 (satu) pak...,1. Apa dasar hukum yang digunakan oleh hakim d...


### Generate Jawaban

In [ ]:
def get_answer(row):
    try:
        response = client.chat.completions.create(
            messages=[
                {"role": "system", "content": "Kamu adalah asisten saya untuk menjawab pertanyaan dalam konteks hukum pidana narkotika."},
                {
                    "role": "user",
                    "content": f"Jawablah pertanyaan dalam bahasa indonesia berdasakan konteks dibawah ini:\n\nText: {row.Context}\n\nQuestions:\n{row.questions}\r"}
            ],
            model="gpt-4o-mini",
            temperature=0.8,
            max_tokens=700,
            top_p=0.8,
            frequency_penalty=0,
            presence_penalty=0
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        print(f"An error occurred: {e}")

df['answer']= df.apply(get_answer, axis=1)
df = df.dropna().reset_index().drop('index',axis=1)
print(df[['answer']].values[0][0])

1. Barang bukti yang ditemukan dalam kasus Terdakwa RAKA ARDI SULISTYO adalah 1 (satu) buah bekas bungkus rokok Luxio berisi 1 (satu) paket tembakau gorilla yang dibungkus plastik klip dengan berat 0,49 gram, serta 1 (satu) pack kertas paper merek ROYO. Total berat narkotika yang terlibat adalah 0,49 gram.

2. Keputusan hakim mengenai hukuman yang dijatuhkan kepada Terdakwa adalah pidana penjara selama 1 (satu) tahun. Masa penangkapan dan penahanan yang telah dijalani oleh Terdakwa akan dikurangkan seluruhnya dari pidana yang dijatuhkan.

3. Barang bukti yang dirampas dan dimusnahkan berdasarkan putusan hakim dalam kasus ini adalah 1 (satu) buah bekas bungkus rokok Luxio berisi 1 (satu) paket tembakau gorilla yang dibungkus plastik klip, serta 1 (satu) pack kertas paper merek ROYO. Selain itu, 1 (satu) buah HP merk OPPO warna Hitam juga dirampas untuk negara.


In [ ]:
df.head()

,no_putusan,lembaga_peradilan,barang_bukti,amar_putusan,tokens,Context,questions,answer
0,640/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah bekas bungkus rokok Luxio berisi...,M E N G A D I L I :; Menyatakan Terdakwa RAKA ...,376,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa barang bukti yang ditemukan dalam kasus...,1. Barang bukti yang ditemukan dalam kasus Ter...
1,641/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket tembakau Gorila yang dibungkus ...,M E N G A D I L I; Menyatakan Terdakwa I. AHMA...,1188,Berdasarkan barang bukti berikut: 1 (satu) pak...,1. Apa dasar hukum yang menjadi acuan dalam me...,1. Dasar hukum yang menjadi acuan dalam menjat...
2,545/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas pinggang warna hitam;; 1 (sa...,MENGADILI:; Menyatakan terdakwa AKIN SUNARYO B...,465,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa dasar keputusan hakim dalam menyatakan ...,1. Dasar keputusan hakim dalam menyatakan terd...
3,586/Pid.Sus/2024/PN Smn,PN SLEMAN,Bekas bungkus rokok merk Camel yang di dalamny...,MENGADILI:; Menyatakan Terdakwa Septian Eko Ca...,699,Berdasarkan barang bukti berikut: Bekas bungku...,1. Apa jenis narkotika yang ditemukan dalam ba...,1. Jenis narkotika yang ditemukan dalam barang...
4,446/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas ransel warna hitam bertulisk...,M E N G A D I L I; Menyatakan Terdakwa I. Cost...,2388,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa alasan hakim memutuskan untuk menjatuhk...,1. Hakim memutuskan untuk menjatuhkan pidana p...


### Simpan Dataset QA

In [ ]:
df.to_csv('datasetnqa.csv', index=False)

In [ ]:
# Simpan Dataset Pada Json

qa_list = []

# Iterasi baris pada dataframe
for index, row in df.iterrows():
  questions = row['questions']
  answer = row['answer']

  qa_list.append({"question": questions, "answer": answer})

#  Konversi list dictionary ke JSON list
json_data = json.dumps(qa_list, indent=4)

# Simpan JSON file
with open('questions_answers.json', 'w') as json_file:
  json_file.write(json_data)

In [ ]:
# Simpan kolom question dan answer dalam satu csv
df[['questions', 'answer']].to_csv('questions_answers.csv', index=False)

## Embedding dataset QA dengan Open AI

In [ ]:
df = pd.read_csv('datasetnqa.csv')

In [ ]:
df.head()

,no_putusan,lembaga_peradilan,barang_bukti,amar_putusan,tokens,Context,questions,answer
0,640/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah bekas bungkus rokok Luxio berisi...,M E N G A D I L I :; Menyatakan Terdakwa RAKA ...,376,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa barang bukti yang ditemukan dalam kasus...,1. Barang bukti yang ditemukan dalam kasus Ter...
1,641/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) paket tembakau Gorila yang dibungkus ...,M E N G A D I L I; Menyatakan Terdakwa I. AHMA...,1188,Berdasarkan barang bukti berikut: 1 (satu) pak...,1. Apa dasar hukum yang menjadi acuan dalam me...,1. Dasar hukum yang menjadi acuan dalam menjat...
2,545/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas pinggang warna hitam;; 1 (sa...,MENGADILI:; Menyatakan terdakwa AKIN SUNARYO B...,465,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa dasar keputusan hakim dalam menyatakan ...,1. Dasar keputusan hakim dalam menyatakan terd...
3,586/Pid.Sus/2024/PN Smn,PN SLEMAN,Bekas bungkus rokok merk Camel yang di dalamny...,MENGADILI:; Menyatakan Terdakwa Septian Eko Ca...,699,Berdasarkan barang bukti berikut: Bekas bungku...,1. Apa jenis narkotika yang ditemukan dalam ba...,1. Jenis narkotika yang ditemukan dalam barang...
4,446/Pid.Sus/2024/PN Smn,PN SLEMAN,1 (satu) buah tas ransel warna hitam bertulisk...,M E N G A D I L I; Menyatakan Terdakwa I. Cost...,2388,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa alasan hakim memutuskan untuk menjatuhk...,1. Hakim memutuskan untuk menjatuhkan pidana p...


### Memisahkan Pertanyaan dan Jawaban setiap Baris

In [ ]:
df_split = pd.DataFrame(columns=['Context', 'Questions', 'Answers'])

# Fungsi untuk memisahkan pertanyaan dan jawaban
def split_questions_answers(context, questions, answers):
    # Pisahkan pertanyaan berdasarkan pola angka diikuti titik dan spasi "1. /2. /3. "
    question_segments = re.split(r'(?<=\d\.\s)', questions)[1:]
    question_segments = [f"{i + 1}. {seg.strip()}" for i, seg in enumerate(question_segments)]

    # Pisahkan jawaban berdasarkan pola angka diikuti titik dan spasi
    answer_segments = re.split(r'(?<=\d\.\s)', answers)[1:]
    answer_segments = [f"{i + 1}. {seg.strip()}" for i, seg in enumerate(answer_segments)]

    # Hilangkan angka di akhir segmen jika ada
    question_segments = [re.sub(r'\d\.$', '', q).strip() for q in question_segments]
    answer_segments = [re.sub(r'\d\.$', '', a).strip() for a in answer_segments]

    # Pastikan jumlah pertanyaan dan jawaban sama
    max_len = max(len(question_segments), len(answer_segments))
    question_segments += [''] * (max_len - len(question_segments))
    answer_segments += [''] * (max_len - len(answer_segments))

    # Gabungkan hasil menjadi satu list
    split_data = []
    for i in range(max_len):
        question = question_segments[i].strip()
        answer = answer_segments[i].strip()
        split_data.append([context, question, answer])
    return split_data

# Iterasi setiap baris data dan proses
for c, q, a in zip(df['Context'], df['questions'], df['answer']):
    split_result = split_questions_answers(c, q, a)
    df_split = pd.concat([df_split, pd.DataFrame(split_result, columns=['Context', 'Questions', 'Answers'])],
                         ignore_index=True)
# Tampilkan hasil
display(df_split)

,Context,Questions,Answers
0,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa barang bukti yang ditemukan dalam kasus...,1. Barang bukti yang ditemukan dalam kasus Ter...
1,Berdasarkan barang bukti berikut: 1 (satu) bua...,2. Apa keputusan hakim mengenai hukuman yang d...,2. Keputusan hakim mengenai hukuman yang dijat...
2,Berdasarkan barang bukti berikut: 1 (satu) bua...,3. Apa saja barang bukti yang dirampas dan dim...,3. Barang bukti yang dirampas dan dimusnahkan ...
3,Berdasarkan barang bukti berikut: 1 (satu) pak...,1. Apa dasar hukum yang menjadi acuan dalam me...,1. Dasar hukum yang menjadi acuan dalam menjat...
4,Berdasarkan barang bukti berikut: 1 (satu) pak...,2. Berapa total berat paket tembakau Gorila ya...,2. Total berat paket tembakau Gorila yang dite...
...,...,...,...
154,Berdasarkan barang bukti berikut: : - 1 (satu)...,2. Pasal berapa dalam Undang-Undang Nomor 35 T...,2. Terdakwa melanggar Pasal 127 ayat (1) huruf...
155,Berdasarkan barang bukti berikut: : - 1 (satu)...,3. Mengapa hakim memutuskan untuk tetap menaha...,3. Hakim memutuskan untuk tetap menahan Terdak...
156,Berdasarkan barang bukti berikut: 1 (satu) pla...,1. Apa barang bukti yang ditemukan dan diduga ...,1. Barang bukti yang ditemukan dan diduga terk...
157,Berdasarkan barang bukti berikut: 1 (satu) pla...,2. Apa alasan yang mendasari hakim untuk menja...,2. Alasan yang mendasari hakim untuk menjatuhk...


In [ ]:
# Memeriksa apakah spliting berjalan dengan baik
df_split.isnull().sum()

,0
Context,0
Questions,0
Answers,0


In [ ]:
# Menyimpan data untuk keperluan lebih lanjut
df_split.to_csv('dataset_QA.csv', index=False)

### Melakukan Embedding pada kolom answer

In [ ]:
df = pd.read_csv('dataset_QA.csv')

In [ ]:
# Melakukan embedding pada kolom answer
def embedding_answer(context):
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=context,
            encoding_format="float"
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"An error occurred: {e}")

df['Embedding'] = df.Answers.apply(embedding_answer)
df.to_csv('split_embedqa.csv', index=False)

In [ ]:
df

,Context,Questions,Answers,Embedding
0,Berdasarkan barang bukti berikut: 1 (satu) bua...,1. Apa barang bukti yang ditemukan dalam kasus...,1. Barang bukti yang ditemukan dalam kasus Ter...,"[0.030590892, -0.041039117, -0.056784738, 0.01..."
1,Berdasarkan barang bukti berikut: 1 (satu) bua...,2. Apa keputusan hakim mengenai hukuman yang d...,2. Keputusan hakim mengenai hukuman yang dijat...,"[-0.021558937, -0.008863621, 0.028588237, 0.05..."
2,Berdasarkan barang bukti berikut: 1 (satu) bua...,3. Apa saja barang bukti yang dirampas dan dim...,3. Barang bukti yang dirampas dan dimusnahkan ...,"[0.021156522, -0.008780684, -0.03496594, 0.014..."
3,Berdasarkan barang bukti berikut: 1 (satu) pak...,1. Apa dasar hukum yang menjadi acuan dalam me...,1. Dasar hukum yang menjadi acuan dalam menjat...,"[-0.03088844, -0.030295469, 0.012317642, 0.038..."
4,Berdasarkan barang bukti berikut: 1 (satu) pak...,2. Berapa total berat paket tembakau Gorila ya...,2. Total berat paket tembakau Gorila yang dite...,"[0.05605842, -0.017753227, -0.02646275, 0.0145..."
...,...,...,...,...
154,Berdasarkan barang bukti berikut: : - 1 (satu)...,2. Pasal berapa dalam Undang-Undang Nomor 35 T...,2. Terdakwa melanggar Pasal 127 ayat (1) huruf...,"[-0.0054817917, -0.026795508, 0.008877852, 0.0..."
155,Berdasarkan barang bukti berikut: : - 1 (satu)...,3. Mengapa hakim memutuskan untuk tetap menaha...,3. Hakim memutuskan untuk tetap menahan Terdak...,"[-0.005480597, -0.0074396385, 0.033742726, 0.0..."
156,Berdasarkan barang bukti berikut: 1 (satu) pla...,1. Apa barang bukti yang ditemukan dan diduga ...,1. Barang bukti yang ditemukan dan diduga terk...,"[0.018630156, -0.0009369451, -0.04097361, 0.04..."
157,Berdasarkan barang bukti berikut: 1 (satu) pla...,2. Apa alasan yang mendasari hakim untuk menja...,2. Alasan yang mendasari hakim untuk menjatuhk...,"[-0.026118191, -0.01317992, 0.02289621, 0.0629..."


### Melakukan embedding pada query

In [ ]:
# Melakukan embedding pada pertanyaan baru (query)
def embedding_query(query):
    try:
        response = client.embeddings.create(
            model="text-embedding-3-small",
            input=query,
            encoding_format="float"
        )
        return response.data[0].embedding
    except Exception as e:
        print(f"An error occurred: {e}")

## Input pertanyaan baru dari user

In [ ]:
query = "Apa saja barang bukti yang ditemukan dan dijadikan dasar dalam putusan narkotika?"

## Algoritma Pencocokan Data

### Perhitungan Cosine

Implementasi algoritma pencocokan pertanyaan baru terhadap jawaban berdasarkan embedding. Skor jarak cosine digunakan untuk menentukan relevansi antara pertanyaan baru dan jawaban yang ada. Jawaban dengan nilai jarak cosine terkecil dianggap paling relevan.

In [ ]:
def rank_distances(query_embedding, embeddings_list):
    distances = [(i, cosine(query_embedding, embedding)) for i, embedding in enumerate(embeddings_list)]
    return sorted(distances, key=lambda x: x[1])

### Hasil jawaban paling relevan

In [ ]:
query_embedding = embedding_query(query)
ranked_results = rank_distances(query_embedding, df["Embedding"])

# Retrieve the top 5 results from ranked_results with respective ranks and answers
top_5_results = [
    {"No": idx + 1, "Rank": rank, "Answer": df["Answers"][i]}
    for idx, (i, rank) in enumerate(ranked_results[:5])
]

# Create a DataFrame for display
print("Pertanyaan: ", query)
top_5_df = pd.DataFrame(top_5_results)
display(top_5_df)

Pertanyaan:  Apa saja barang bukti yang ditemukan dan dijadikan dasar dalam putusan narkotika?


,No,Rank,Answer
0,1,0.238908,"2. Barang bukti yang disita, seperti tas ranse..."
1,2,0.256997,3. **Pengaruh barang bukti terhadap putusan ha...
2,3,0.278808,1. Barang bukti yang ditemukan dalam kasus Ter...
3,4,0.286243,1. Barang bukti yang ditemukan dalam kasus ter...
4,5,0.289931,3. Barang bukti yang dihadirkan dalam persidan...


# Pengujian model QA dengan pertanyaan lain

## Uji pertanyaan 1

In [ ]:
query = "Apa dasar hukum yang menjadi acuan dalam menjatuhkan pidana kepada kasus ppenyalahgunaan narkoba?"

query_embedding = embedding_query(query)
ranked_results = rank_distances(query_embedding, df["Embedding"])

# Retrieve the top 5 results from ranked_results with respective ranks and answers
top_5_results = [
    {"No": idx + 1, "Rank": rank, "Answer": df["Answers"][i]}
    for idx, (i, rank) in enumerate(ranked_results[:5])
]

# Create a DataFrame for display
print("Pertanyaan: ", query)
top_5_df = pd.DataFrame(top_5_results)
display(top_5_df)

Pertanyaan:  Apa dasar hukum yang menjadi acuan dalam menjatuhkan pidana kepada kasus ppenyalahgunaan narkoba?


,No,Rank,Answer
0,1,0.200826,1. Dasar hukum yang menjadi acuan dalam menjat...
1,2,0.264983,1. Dasar hukum yang digunakan hakim dalam memu...
2,3,0.289602,1. Dasar hukum yang digunakan oleh hakim dalam...
3,4,0.303524,2. Dasar hukum yang digunakan oleh hakim dalam...
4,5,0.306024,1. Dasar hukum bagi hakim untuk menjatuhkan pi...


## Uji pertanyaan 2

In [ ]:
query = "Berapa lama hukuman penjara yang dijatuhkan kepada kasus penyalahgunaan narkoba?"

query_embedding = embedding_query(query)
ranked_results = rank_distances(query_embedding, df["Embedding"])

# Retrieve the top 5 results from ranked_results with respective ranks and answers
top_5_results = [
    {"No": idx + 1, "Rank": rank, "Answer": df["Answers"][i]}
    for idx, (i, rank) in enumerate(ranked_results[:5])
]

# Create a DataFrame for display
print("Pertanyaan: ", query)
top_5_df = pd.DataFrame(top_5_results)
display(top_5_df)

Pertanyaan:  Berapa lama hukuman penjara yang dijatuhkan kepada kasus penyalahgunaan narkoba?


,No,Rank,Answer
0,1,0.273461,2. Hukuman penjara yang dijatuhkan kepada masi...
1,2,0.296075,1. Hakim memutuskan untuk menjatuhkan pidana p...
2,3,0.335953,2. Alasan yang mendasari hakim untuk menjatuhk...
3,4,0.354596,3. Pasal yang dilanggar oleh terdakwa dalam ti...
4,5,0.356224,2. Proses penjatuhan pidana terhadap terdakwa ...


## Uji pertanyaan 3

In [ ]:
query = "Apa saja barang tersangka yang disita dan dirampas oleh negara? "

query_embedding = embedding_query(query)
ranked_results = rank_distances(query_embedding, df["Embedding"])

# Retrieve the top 5 results from ranked_results with respective ranks and answers
top_5_results = [
    {"No": idx + 1, "Rank": rank, "Answer": df["Answers"][i]}
    for idx, (i, rank) in enumerate(ranked_results[:5])
]

# Create a DataFrame for display
print("Pertanyaan: ", query)
top_5_df = pd.DataFrame(top_5_results)
display(top_5_df)

Pertanyaan:  Apa saja barang tersangka yang disita dan dirampas oleh negara? 


,No,Rank,Answer
0,1,0.385730,4. Keputusan hakim mengenai barang bukti adala...
1,2,0.389271,3. Barang bukti yang dirampas untuk dimusnahka...
2,3,0.390939,3. Barang bukti yang dirampas untuk dimusnahka...
3,4,0.391131,3. Barang-barang yang dirampas untuk dimusnahk...
4,5,0.392639,3. Barang bukti yang telah dihadirkan dalam pe...
